1. Installing Dependencies


In [1]:
# %pip install tensorflow==2.6 opencv-python mediapipe==0.9.1.0 scikit-learn matplotlib

In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import time
import os
import mediapipe as mp

4. Setup Folders for Collection

In [48]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data_ikaw_2') 

# Actions that we try to detect
actions = np.array(['ako'])
# actions = np.array(['ako',  'ikaw', 'maganda', 'magandang umaga', 'hi', 'hindi', 'oo', 'salamat', 'bakit', 'kamusta'])

# Thirty videos worth of data
no_sequences = 25

# Videos are going to be 30 frames in length
sequence_length = 30

In [12]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

5. Collect Videos and Store them under videos/signed_videos

In [54]:
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

directory = "O121"
if not os.path.exists(directory):
    os.makedirs(directory)

# Set mediapipe model 
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
# Loop through actions
for action in actions:
    # Loop through sequences aka videos
    ret, frame = cap.read()
    cv2.putText(frame, 'Collecting frames in 1 second...', (120,200), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
    cv2.imshow('OpenCV Feed', frame)
    cv2.waitKey(1000)
    for sequence in range(no_sequences):
        out = cv2.VideoWriter(f'{directory}/{sequence}.avi', cv2.VideoWriter_fourcc(*'MJPG'), 20.0, (640, 480))
        # Loop through video length aka sequence length
        for frame_num in range(sequence_length):

            # Read feed
            ret, frame = cap.read()
                
            cv2.namedWindow('OpenCV Feed', cv2.WINDOW_NORMAL)
            cv2.resizeWindow('OpenCV Feed', 900, 600)
            
            out.write(frame)

            # Apply wait logic
            if frame_num == 0: 
                cv2.putText(frame, 'Collecting frames', (120,200), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                cv2.putText(frame, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,35), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                # Show to screen
                cv2.imshow('OpenCV Feed', frame)
                cv2.waitKey(500)
            else: 
                cv2.putText(frame, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,32), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                # Show to screen
                cv2.imshow('OpenCV Feed', frame)
                
            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
            
                    
cap.release()
out.release()
cv2.destroyAllWindows()

[ WARN:0@9368.891] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_d9lyif19nl/croot/opencv-suite_1676472756314/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created

(<unknown>:74690): GStreamer-CRITICAL **: 20:49:14.046: gst_element_make_from_uri: assertion 'gst_uri_is_valid (uri)' failed
[ WARN:0@9371.452] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_d9lyif19nl/croot/opencv-suite_1676472756314/work/modules/videoio/src/cap_gstreamer.cpp (2180) open OpenCV | GStreamer warning: cannot link elements

(<unknown>:74690): GStreamer-CRITICAL **: 20:49:15.616: gst_element_make_from_uri: assertion 'gst_uri_is_valid (uri)' failed
[ WARN:0@9373.021] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_d9lyif19nl/croot/opencv-suite_1676472756314/work/modules/videoio/src/cap_gstreamer.cpp (2180) open OpenCV | GStreamer warning: cannot link elements

(<unknown

KeyboardInterrupt: 

In [26]:
cap.release()
cv2.destroyAllWindows()

6. Arrange the folders: If there are already existing dataset on MP_Data similar to MP_Data_2, then run this cell to transfer the videos from MP_Data_2 to MP_Data. This is to avoid overwriting the existing dataset.

In [45]:
import os
import shutil

source_folder = "../videos/signed_videos_extra"
destination_folder = "../videos/signed_videos"

for folder_name in os.listdir(source_folder):
    source_path = os.path.join(source_folder, folder_name)
    destination_path = os.path.join(destination_folder, folder_name)
    if os.path.isdir(source_path) and os.path.exists(destination_path):
        max_number = max([int(file_name.split(".")[0]) for file_name in os.listdir(destination_path) if file_name.endswith(".mp4")])
        for file_name in os.listdir(source_path):
            if file_name.endswith(".mp4"):
                max_number += 1
                new_file_name = f"{max_number}.mp4"
                source_file = os.path.join(source_path, file_name)
                destination_file = os.path.join(destination_path, new_file_name)
                shutil.move(source_file, destination_file)
                
# This code will compare the two folders src & destination. If there is a similar name folder in both src & destination, then it will move all .mp4 files from the src folder to the folder from destination. The name convention will be followed when the .mp4 files are moved. The videos will be numbered after 14.mp4 so the next video is 15.mp4 and the next video transferred is 16.mp4 and so on.

In [55]:
import os
import subprocess

for root, dirs, files in os.walk('../videos/signed_videos_extra/'):
    for file in files:
        if file.endswith('.avi'):
            avi_file = os.path.join(root, file)
            base = os.path.splitext(file)[0]
            mp4_file = os.path.join(root, f'{base}.mp4')
            subprocess.run(['ffmpeg', '-i', avi_file, '-c:v', 'libx264', '-c:a', 'aac', '-movflags', 'faststart', mp4_file])

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with clang version 12.0.0
  configuration: --prefix=/Users/ktietz/demo/mc3/conda-bld/ffmpeg_1628925491858/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=arm64-apple-darwin20.0.0-clang --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57

In [56]:
import os

for root, dirs, files in os.walk('../videos/signed_videos_extra/'):
    for file in files:
        if file.endswith('.avi'):
            os.remove(os.path.join(root, file))
